Lab 2: Text Classification
=============

In this problem set, you will build a system for automatically classifying song lyrics comments by era. You will:

- Do some basic text processing, tokenizing your input and converting it into a bag-of-words representation
- Build a machine learning classifier based on the generative model, using Naive Bayes
- Evaluate your classifiers and examine what they have learned
- Build a logistic regression classifier (discriminative model) using scikit-learn

Total Points: 120 points

# 0. Setup

In order to develop this assignment, you will need [python 3.6](https://www.python.org/downloads/) and the following libraries. Most if not all of these are part of [anaconda](https://www.continuum.io/downloads), so a good starting point would be to install that.

- [jupyter](http://jupyter.readthedocs.org/en/latest/install.html)
- numpy (This will come if you install scipy like above, but if not install separately)
- [matplotlib](http://matplotlib.org/users/installing.html)
- [nosetests](https://nose.readthedocs.org/en/latest/)
- [pandas](http://pandas.pydata.org/) Dataframes

Here is some help on installing packages in python: https://packaging.python.org/installing/. You can use ```pip --user``` to install locally without sudo.

## About this assignment

- This is a Jupyter notebook. You can execute cell blocks by pressing control-enter.
- All of your coding will be in the python file ```lab2.py```. 
- The file ```tests/tests_visible.py``` contains the Gradescope autograder unit tests that will be available for you to run locally. You should run them as you work on the assignment to see that you're on the right track. You are free to look at their source code, if that helps. You can run the tests by running ```python run_tests.py``` or ```python run_tests.py -j``` for more description. 
- You may want to add more tests, but that is completely optional. 
- **To submit this assignment, submit ```lab2.py``` on Gradescope.**

## Important Instructions for this assignment

- Since each test case takes about 1 minute to run individually and the collective test suite takes about 20-30 minutes to run in its entirety, we recommend that when you implement an individual function you can comment out the remaining test case functions in tests/test_visible.py and only keep the corresponding test case and the def Setup(self) (i.e first function) in an uncommented state.
- We estimate that your completed code should be able to complete running on all the test cases in about 20-30 minuetes. However, if your code takes longer to run, follow the next bullet point.
- The gradescope autograder has a runtime limit of 40 minutes, so if your code times out with the autograder unable to run on all the test cases, then we have a solution for you. The ECE 365 Gradescope page has two assignments: **NLP Lab 2 Code** and **NLP Lab 2 Screenshot**. You will submit your code to **NLP Lab 2 Code**, which will run the autograder. If your code is unable to finish running on all the test cases before timeout, then you would need to submit a screenshot of the local test case output on the **NLP Lab 2 Screenshot** assignment. First run ```python run_tests.py -j``` in the assignment directory and then take a screenshot of the  prompt which shows your final score. An example screenshot is shown below.
- You only need to submit the screenshot if the gradescope autograder is unable to run your code on all the test cases. Submitting your code in **NLP Lab 2 Code**  is a requirement and you will not recieve any credit from your screenshot submission if you have not submitted your code. 

![title](screenshot.png)

In [ ]:
import sys
from importlib import reload

In [ ]:
import lab2

In [ ]:
print('My Python version')

print('python: {}'.format(sys.version))

In [ ]:
import nose

import pandas as pd
import numpy as np
import scipy as sp
import matplotlib
import matplotlib.pyplot as plt

import torch
from torch.autograd import Variable
from torch import optim

%matplotlib inline

In [ ]:
print('My library versions')

print('pandas: {}'.format(pd.__version__))
print('numpy: {}'.format(np.__version__))
print('scipy: {}'.format(sp.__version__))
print('matplotlib: {}'.format(matplotlib.__version__))
print('nose: {}'.format(nose.__version__))
print('torch: {}'.format(torch.__version__))

To test whether your libraries are the right version, run:

`nosetests tests/test_environment.py`

In [ ]:
# use ! to run shell commands in notebook
! nosetests tests/test_environment.py

# 1. Preprocessing

**Total: 20 points**

Read the data into a dataframe

In [ ]:
df_train = pd.read_csv('lyrics-train.csv')

A dataframe is a structured representation of your data. You can preview a dataframe using `head()`

In [ ]:
df_train.head()

## Bags of words

Your first task is to convert the text to a bag-of-words representation. For this data, a lot of the preprocessing is already done: the text is lower-cased, and punctuation is removed. You need only create a `counter` for each instance.

- **Deliverable 1.1**: Complete the function `lab2.bag_of_words`. (5 points)
- **Test**: `tests/test_visible.py:test_d1_1_bow`

In [ ]:
# run this block to update the notebook as you change the preproc library
reload(lab2);

In [ ]:
y_tr,x_tr = lab2.read_data('lyrics-train.csv',preprocessor=lab2.bag_of_words)
y_dv,x_dv = lab2.read_data('lyrics-dev.csv',preprocessor=lab2.bag_of_words)

In [ ]:
y_te,x_te = lab2.read_data('lyrics-test-hidden.csv',preprocessor=lab2.bag_of_words)

## Unseen words

One challenge for classification is that words will appear in the test data that do not appear in the training data. Compute the number of words that appear in `lyrics-dev.csv`, but not in `lyrics-train.csv`. To do this, implement the following deliverables:

- **Deliverable 1.2**: implement `lab2.compute_oov`, returning a list of words that appear in one list of bags-of-words, but not another. Also implement `lab2.aggregate_counts` (10 points)
- **Tests**: `tests/test_visible.py:test_d1_3a_oov` and `tests/test_visible.py:test_d1_2agg`

In [ ]:
from collections import Counter

In [ ]:
reload(lab2);

To write fast code, you can find bottlenecks using the %%timeit cell magic. (The following line will run for about 5 mins.)

In [ ]:
%%timeit
lab2.aggregate_counts(x_tr)

In [ ]:
counts_dv = lab2.aggregate_counts(x_dv)

You can see the most common items in a counter by calling `counts.most_common()`:

In [ ]:
counts_dv.most_common(5)

In [ ]:
counts_tr = lab2.aggregate_counts(x_tr)

In [ ]:
len(lab2.compute_oov(counts_dv,counts_tr))

In [ ]:
len(lab2.compute_oov(counts_tr,counts_dv))

In [ ]:
lab2.oov_rate(counts_dv,counts_tr)

30% of the words in the dev set do not appear in the training set.

## Pruning the vocabulary

Let's prune the vocabulary to include only words that appear at least ten times in the training data.

- **Deliverable 1.3:** Implement `lab2.prune_vocabulary` (5 points)
- **Test**: `tests/test_visible.py:test_d1_4_prune`

In [ ]:
reload(lab2);

In [ ]:
x_tr_pruned, vocab = lab2.prune_vocabulary(counts_tr,x_tr,10)
x_dv_pruned, _ = lab2.prune_vocabulary(counts_tr,x_dv,10)
x_te_pruned, _ = lab2.prune_vocabulary(counts_tr,x_te,10)

In [ ]:
len(vocab)

In [ ]:
i = 94
print(len(x_dv[i]),len(x_dv_pruned[i]))
print(sum(x_dv[i].values()),sum(x_dv_pruned[i].values()))

# 2. Linear classification

Now we'll show you how to implement the linear classification rule, $\hat{y} = \text{argmax}_y \theta^{\top} f(x,y)$.

You will use these functions in all classifiers in this assignment.

**Total: 10 points** 

In [ ]:
reload(lab2);

The feature function vector $f(x,y)$ can be viewed as a dict, in which the values are counts, and the keys are tuples $(y,x_j)$, where $y$ is a label and $x_j$ is a base feature. Note that we must also include the offset feature, ```lab2.OFFSET```. Desired output is shown below:

In [ ]:
fv = lab2.make_feature_vector({'test':1,'case':2},'1980s')

In [ ]:
print(fv)

Let's compute the entire set of labels.

In [ ]:
labels = set(y_tr) #figure out all possible labels
print(labels)

Now we implement the prediction rule, $\hat{y} = \text{argmax}_y \theta^{\top} f(x,y)$.

The output should be:

- A predicted label
- The scores of all labels

You can test this function using these simple hand-crafted weights.

In [ ]:
from collections import defaultdict
reload(lab2)

In [ ]:
# weight vectors must be defaultdicts
theta_hand = defaultdict(float,
                         {('2000s','money'):0.1,
                          ('2000s','name'):0.2,
                          ('1980s','tonight'):0.1,
                          ('2000s','man'):0.1,
                          ('1990s','fly'):0.1,
                          ('pre-1980',lab2.OFFSET):0.1
                         })

In [ ]:
lab2.predict(x_tr_pruned[0],theta_hand,labels)

Now let's see how good these weights are, by evaluating on the dev set.

In [ ]:
reload(lab2);

In [ ]:
# this applies your predict function to all the instances in ```x_dv```
y_hat = lab2.predict_all(x_dv_pruned,theta_hand,labels)
print(lab2.acc(y_hat,y_dv))

# 3. Naive Bayes

You'll now implement a Naive Bayes classifier in this section.

**Total: 45 points**

In [ ]:
reload(lab2);

- **Deliverable 3.1**: (warmup) implement ```get_corpus_counts``` in ```lab2.py```. (5 points)
- **Test**: `tests/test_visible.py:test_d3_1_corpus_counts`

This function should compute the word counts for a given label.

In [ ]:
eighties_counts = lab2.get_corpus_counts(x_tr_pruned,y_tr,"1980s");
print(eighties_counts['today'])
print(eighties_counts['yesterday'])

- **Deliverable 3.2**: Implement ```estimate_pxy``` in ```lab2.py```. (15 points)
- **Test**: `tests/test_visible.py:test_d3_2_pxy`

This function should compute the *smoothed* multinomial distribution $\log P(x \mid y)$ for a given label $y$.
Note that this function takes the vocabulary as an argument. You have to assign a probability even for words that do not appear in documents with label $y$, if they are in the vocabulary.

Hint: You can use ```get_corpus_counts``` in this function if you want to, but you don't have to.

In [ ]:
log_pxy = lab2.estimate_pxy(x_tr_pruned,y_tr,"1980s",0.1,vocab)

Probabilities must sum to one! (or very close)

In [ ]:
sum(np.exp(list(log_pxy.values())))

Let's look at the log-probabilities of the words from the hand-tuned weights

In [ ]:
print({word:log_pxy[word] for (_,word),weight in theta_hand.items() if weight>0})

In [ ]:
log_pxy_more_smooth = lab2.estimate_pxy(x_tr_pruned,y_tr,"1980s",1000,vocab)

In [ ]:
print({word:log_pxy_more_smooth[word] for (_,word),weight in theta_hand.items() if weight>0})

- **Deliverable 3.3**: Now you are ready to implement ```estimate_nb``` in ```lab2.py```. (15 points)
- **Test**: `tests/test_visible.py:test_d3_3a_nb`



- The goal is that the score given by ```lab2.predict``` is equal to the joint probability $P(x,y)$, as described in the notes. Therefore, make sure your return output can be feed into ```lab2.predict```. 
- Don't forget the offset feature, whose weights should be set to the prior $\log P(y)$.
- The log-probabilities for the offset feature should not be smoothed.
- You can call the functions you have defined above, but you don't have to.

In [ ]:
reload(lab2);
theta_nb = lab2.estimate_nb(x_tr_pruned,y_tr,0.1)

Let's predict for a single instance. 

In [ ]:
lab2.predict(x_tr_pruned[155],theta_nb,labels)

In [ ]:
aaa = lab2.predict(x_tr_pruned[155],theta_nb,labels)
print(aaa)
aaa = lab2.predict(x_tr_pruned[55],theta_nb,labels)
print(aaa)

Let's predict for all instances of the development set. 

In [ ]:
y_hat = lab2.predict_all(x_dv_pruned,theta_nb,labels)
print(lab2.acc(y_hat,y_dv))

In [ ]:
# this block shows how we write and read predictions for evaluation
lab2.write_predictions(y_hat,'nb-dev.preds')
y_hat_dv = lab2.read_predictions('nb-dev.preds')
lab2.acc(y_hat_dv,y_dv)

In [ ]:
# execute this block to write predictions for the test set
y_hat = lab2.predict_all(x_te_pruned,theta_nb,labels)
lab2.write_predictions(y_hat,'nb-test.preds')

- **Deliverable 3.4**: Write a function in ```lab2.py``` called ```find_best_smoother```, which finds the smoothing value that gives best performance on the dev data.  (5 points)
- **Test**: `tests/test_visible.py:test_d3_4a_nb_best`

Your function should be trying at least the following values in `vals` below.

Then, using this smoothing value, run your Naive Bayes classifier on the test set, and output the results.

In [ ]:
vals = np.logspace(-3,2,11)
print(vals)

In [ ]:
reload(lab2);
best_smoother, scores = lab2.find_best_smoother(x_tr_pruned,y_tr,x_dv_pruned,y_dv,vals)

In [ ]:
plt.semilogx(list(scores.keys()),list(scores.values()),'o-');
plt.xlabel('smoothing')
plt.ylabel('dev set accuracy');

**Reflect:**

- what might explain the dramatic drop in accuracy when the smoothing is increased from $10$ to $30$?
- before you check, predict whether the accuracy will continue to significantly drop if you further increase the smoothing to $10000$.

**Your Answer Here**: 

Save the best parameters for later comparison. 

In [ ]:
theta_nb = lab2.estimate_nb(x_tr_pruned,y_tr,best_smoother)
y_hat = lab2.predict_all(x_te_pruned,theta_nb,labels)
lab2.write_predictions(y_hat,'nb-best-test.preds')

# 4. Logistic regression

You will implement logistic regression in scikit-learn.

**Total: 15 points**


## 4.1 Converting data to numpy

Numpy is a package for numerical computing in python.

You will need to convert your bag-of-words list of counters to a numpy array. 

- **Deliverable 4.1**: Implement `lab2.py:make_numpy()` (5 points)
- **Test**: `tests/test_visible.py:test_d4_1_numpy`
- **Hint**: one approach is to start with `numpy.zeros((height,width))`, and then fill in the cells by iterating through the bag-of-words list

In [ ]:
X = np.zeros((4,2))
print(X)

In [ ]:
X[1,1] = -1
X[2,0] = 1.5
print(X)

In [ ]:
reload(lab2);

In [ ]:
X_tr = lab2.make_numpy(x_tr_pruned,vocab)
X_dv = lab2.make_numpy(x_dv_pruned,vocab)
X_te = lab2.make_numpy(x_te_pruned,vocab)

In [ ]:
label_set = sorted(list(set(y_tr)))
print(label_set)

In [ ]:
Y_tr = np.array([label_set.index(y_i) for y_i in y_tr])
Y_dv = np.array([label_set.index(y_i) for y_i in y_dv])

In [ ]:
len(set(Y_tr))

## 4.2 Building a logistic regression model

Import the model you want to use and make an instance of the Model.

In [ ]:
from sklearn.linear_model import LogisticRegression
scikit_log_reg = LogisticRegression()

Logistic Regression Model training.

In [ ]:
logisticRegr=scikit_log_reg.fit(X_tr, Y_tr)

Get accuracy of training data and dev data. 

accuracy is defined as:

(fraction of correct predictions): correct predictions / total number of data points

In [ ]:
train_acc = logisticRegr.score(X_tr, Y_tr)
dev_acc = logisticRegr.score(X_dv, Y_dv)

print(train_acc)
print(dev_acc)

**Deliverable 4.2**
The noisy progress of the loss and dev set accuracy suggests that something is wrong with our training hyperparameters. Tune the ```LogisticRegression``` parameters until you can get to a dev set accuracy of at least 0.5. You may find a set of tunable parameters here: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html 
Complete lab2.better_model function
(10 points)

In [ ]:
reload(lab2);

In [ ]:
scikit_log_reg = lab2.better_model()
logisticRegr=scikit_log_reg.fit(X_tr, Y_tr)
train_acc = logisticRegr.score(X_tr, Y_tr)
dev_acc = logisticRegr.score(X_dv, Y_dv)
print(train_acc)
print(dev_acc)

In [ ]:
### BEGIN HIDDEN TESTS
scikit_log_reg = lab2.better_model()
logisticRegr=scikit_log_reg.fit(X_tr, Y_tr)
dev_acc = logisticRegr.score(X_dv, Y_dv)
assert dev_acc >= 0.50
### END HIDDEN TESTS

In [ ]:
Y_hat_te = logisticRegr.predict(X_te)
np.save('logreg-es-test.preds.npy', np.array(Y_hat_te))

# 5. Feature analysis

**Total: 20 points**


## 5.1 Top Features for Logistic Regression

**Deliverable 5.1**: Implement ```get_top_features_LR``` to output the k most indicative features (**highest features weights**) and the k least indicative features (**lowest features weights**) for each label. (10 points)

**Hint**: ```scikit_log_reg.coef_``` is the coefficient of the features. 

Let's load the vanilla LR model for comparison. 

In [ ]:
scikit_log_reg = LogisticRegression()
logisticRegr=scikit_log_reg.fit(X_tr, Y_tr)

In [ ]:
reload(lab2);

In [ ]:
print(lab2.get_top_features_LR(scikit_log_reg, vocab,label_set,'pre-1980',k=10))
print(lab2.get_top_features_LR(scikit_log_reg, vocab,label_set,'1980s',k=10))
print(lab2.get_top_features_LR(scikit_log_reg, vocab,label_set,'1990s',k=10))
print(lab2.get_top_features_LR(scikit_log_reg, vocab,label_set,'2000s',k=10))

In [ ]:
### BEGIN HIDDEN TESTS
scikit_log_reg = LogisticRegression()
logisticRegr=scikit_log_reg.fit(X_tr, Y_tr)
assert set(lab2.get_top_features_LR(scikit_log_reg, vocab,label_set,'pre-1980',k=10)[0]) == set(['lord', 'boogie', 'very', 'feelin', 'darling', 'dancing', 'till', 'mornin', 'fool', 'percussion'])
assert set(lab2.get_top_features_LR(scikit_log_reg, vocab,label_set,'pre-1980',k=10)[1]) == set(['step', 'under', 'meant', 'runaway', 'perfect', 'yo', 'open', 'front', 'body', 'hit'])

assert set(lab2.get_top_features_LR(scikit_log_reg, vocab,label_set,'1980s',k=10)[0]) == set(['wall', 'america', 'standing', 'tumble', 'poison', 'shout', 'chance', 'heat', 'cut', 'took'])
assert set(lab2.get_top_features_LR(scikit_log_reg, vocab,label_set,'1980s',k=10)[1]) == set(['floor', 'hes', 'god', 'percussion', 'thinkin', 'finally', 'window', 'mama', 'lord', 'sing'])

assert set(lab2.get_top_features_LR(scikit_log_reg, vocab,label_set,'1990s',k=10)[0]) == set(['hit', 'yo', 'cuz', 'saw', 'dick', 'cradle', 'front', 'push', 'needed', 'rush'])
assert set(lab2.get_top_features_LR(scikit_log_reg, vocab,label_set,'1990s',k=10)[1]) == set(['dancing', 'second', 'chance', 'born', 'use', 'those', 'pretty', 'meaning', 'today', 'other'])

assert set(lab2.get_top_features_LR(scikit_log_reg, vocab,label_set,'2000s',k=10)[0]) == set(['wit', 'shut', 'shorty', 'club', 'three', 'jeans', 'side', 'ass', 'full', 'bitch'])
assert set(lab2.get_top_features_LR(scikit_log_reg, vocab,label_set,'2000s',k=10)[1]) == set(['lovin', 'rhythm', 'hip', 'lover', 'must', 'honey', 'boogie', 'woman', 'youve', 'fool'])
### END HIDDEN TESTS

## 5.2 Top Features for Naive Bayes


**Deliverable 5.2**: Implement ```get_top_features_NB``` to output the k most indicative features (**highest features weights**) and the k least indicative features (**lowest features weights**) for each label. (10 points)

In [ ]:
reload(lab2);

In [ ]:
print(lab2.get_top_features_NB(theta_nb, label_set,'pre-1980',k=10))
print(lab2.get_top_features_NB(theta_nb, label_set,'1980s',k=10))
print(lab2.get_top_features_NB(theta_nb, label_set,'1990s',k=10))
print(lab2.get_top_features_NB(theta_nb, label_set,'2000s',k=10))

In [ ]:
### BEGIN HIDDEN TESTS
theta_nb = lab2.estimate_nb(x_tr_pruned,y_tr,best_smoother)
assert set(lab2.get_top_features_NB(theta_nb, label_set,'pre-1980',k=10)[0]) == set(['you', 'the', 'i', 'to', 'and', 'a', 'me', 'my', 'it', 'love'])
# assert set(get_top_features_NB(theta_nb, label_set,'pre-1980',k=10)[1]) == set(['master', 'wishful', 'killin', 'benefit', 'zono', 'muzik', 'mewhy', 'overall', 'animal', 'skeet'])

assert set(lab2.get_top_features_NB(theta_nb, label_set,'1980s',k=10)[0]) == set(['you', 'the', 'i', 'to', 'me', 'a', 'and', 'it', 'my', 'love'])
# assert set(get_top_features_NB(theta_nb, label_set,'1980s',k=10)[1]) == set(['lamborghini', 'yeahthe', 'wishful', 'benefit', 'babei', 'zono', 'overall', 'billion', 'fiend', 'skeet'])

assert set(lab2.get_top_features_NB(theta_nb, label_set,'1990s',k=10)[0]) == set(['you', 'i', 'the', 'to', 'me', 'and', 'a', 'it', 'my', 'your'])
# assert set(get_top_features_NB(theta_nb, label_set,'1990s',k=10)[1]) == set(['ladada', 'toot', 'spotlights', 'reverse', 'zono', 'muzik', 'overall', 'tho', 'billion', 'skeet'])

assert set(lab2.get_top_features_NB(theta_nb, label_set,'2000s',k=10)[0]) == set(['you', 'i', 'the', 'me', 'and', 'to', 'a', 'it', 'my', 'in'])
# assert set(get_top_features_NB(theta_nb, label_set,'2000s',k=10)[1]) == set(['eternal', 'shiver', 'stepper', 'escapade', 'jojo', 'tambourine', 'dop', 'wishful', 'total', 'muzik'])
### END HIDDEN TESTS

**Reflect:**

- Compare the development dataset accuracy of LR and NB, which model do you think is better? 
- Given those indicative features of LR and NB, which model do you think is better? 
- You may read https://medium.com/@sangha_deb/naive-bayes-vs-logistic-regression-a319b07a5d4c for more information on a comparison between discriminative and generative models. 

**Your Answer Here**: 

# 6. Precision, Recall, and F1

Besides accuracy, systems in natural language processing are evaluated using precision, recall, and F1. Such measures are essential when evaluating on an unbalanced dataset in terms of classes (labels). 

**Total: 10 points**

## Confusion Matrix

A confusion matrix is a table that is often used to describe the performance of a classification model (or "classifier") on a set of data for which the true values are known. 

In this section, we show one python packages (Seaborn) for making confusion matrixes.

In [ ]:
from sklearn import metrics
import seaborn as sns

predictions = logisticRegr.predict(X_dv)
cm = metrics.confusion_matrix(Y_dv, predictions)

In [ ]:
plt.figure()
ax = sns.heatmap(cm, annot=True, fmt=".1f", linewidths=1, square = True, cmap = 'Blues_r');
ax.set_ylim(0 ,4)
plt.ylabel('Actual label');
plt.xlabel('Predicted label');
all_sample_title = 'Accuracy Score: {0:.4f}'.format(dev_acc)
plt.title(all_sample_title, size = 15);
plt.show();

**Reflect**: What do you observe on the above confusion matrix? If you are the leading manager for this team project, which portion of the data would you ask your team to focus on? 

**Your Answer Here**: 

## Precision, Recall, and F1

Write a function below that takes in a predicted labels 'Y_hat' and gold labels 'Y', and returns the precision, recall, and F1 for each label.

F1 is the harmonic mean of precision and recall. F1 = 2 * (precision * recall) / (precision + recall)

(10 points)

In [ ]:
print(lab2.get_PRF(predictions, Y_dv, label_set, 'pre-1980'))
print(lab2.get_PRF(predictions, Y_dv, label_set, '1980s'))
print(lab2.get_PRF(predictions, Y_dv, label_set, '1990s'))
print(lab2.get_PRF(predictions, Y_dv, label_set, '2000s'))

In [ ]:
label_set

In [ ]:
### BEGIN HIDDEN TESTS
scikit_log_reg = LogisticRegression()
logisticRegr=scikit_log_reg.fit(X_tr, Y_tr)
predictions = logisticRegr.predict(X_dv)
a,b,c = lab2.get_PRF(predictions, Y_dv, label_set, 'pre-1980')
assert abs(a-0.5078125) < 0.01
assert abs(b-0.5241935483870968) < 0.01
assert abs(c-0.5158730158730158) < 0.01

a,b,c = lab2.get_PRF(predictions, Y_dv, label_set, '1980s')
assert abs(a-0.32967032967032966) < 0.01
assert abs(b-0.28846153846153844) < 0.01
assert abs(c-0.30769230769230765) < 0.01

a,b,c = lab2.get_PRF(predictions, Y_dv, label_set, '1990s')
assert abs(a-0.391304347826087) < 0.01
assert abs(b-0.37894736842105264) < 0.01
assert abs(c-0.3850267379679144) < 0.01

a,b,c = lab2.get_PRF(predictions, Y_dv, label_set, '2000s')
assert abs(a-0.6258992805755396) < 0.01
assert abs(b-0.6850393700787402) < 0.01
assert abs(c-0.6541353383458647) < 0.01
### END HIDDEN TESTS